In [7]:
import pandas as pd
from pymongo import MongoClient

In [18]:
def _connect_mongo():
    return MongoClient("mongodb://localhost:27017/")

def read_mongo(db, collection):
    client = _connect_mongo()
    database = client[db]
    col = database[collection]

    df =  pd.DataFrame(list(col.find({})))
    return df

if __name__ == '__main__':
    df = read_mongo('MedGraph', 'Dataset2000Entries')
    df.to_csv('../data/dataset2000entries.csv', index=False)

In [57]:
dataframe = pd.read_csv('../data/dataset2000entries.csv')

# Connect to mongodb and create a new database
client = MongoClient("mongodb://localhost:27017/")
database = client["MedGraph"]

# Upload columns lists as mongodb collections
dataframe["authors"] = dataframe["authors"].apply(lambda x: x.split(", "))
collection = database["Authors"]
for authors in dataframe["authors"]:
    for author in authors:
        collection.insert_one({"author": author})

dataframe["mesh_terms"] = dataframe["mesh_terms"].apply(lambda x: x.split("; "))
collection = database["MeshTerms"]
for terms in dataframe["mesh_terms"]:
    for term in terms:
        _id = term.split(":")[0]
        try:
            collection.insert_one({"_id": _id, "term_name" : term.split(":")[1]})
        except: continue


dataframe["Areas"] = dataframe["Areas"].apply(lambda x: x.split("; "))
collection = database["Areas"]
areas_set = set()
for areas in dataframe["Areas"]:
    for area in areas:
        areas_set.add(area)
        
for area in areas_set:
    collection.insert_one({"area": area})

        
dataframe["Categories"] = dataframe["Categories"].apply(lambda x: x.split("; "))
collection = database["Categories"]
categories_set = set()
for categories in dataframe["Categories"]:
    for category in categories:
        if " (Q1)" in category: category = category.replace(" (Q1)", "")
        elif " (Q2)" in category: category = category.replace(" (Q2)", "")
        elif " (Q3)" in category: category = category.replace(" (Q3)", "")
        elif " (Q4)" in category: category = category.replace(" (Q4)", "")
        
        categories_set.add(category)
        
for category in categories_set:
    collection.insert_one({"category": category})


# dataframe["keywords"] = dataframe["keywords"].apply(lambda x: x.split("; "))